In [3]:
from langchain_openai import AzureChatOpenAI
from util.config import load_configuration

PROP_FILE = 'app.properties'
configs = load_configuration(PROP_FILE)

API_TYPE = configs.get("API_TYPE").data
API_VERSION = configs.get("API_VERSION").data
API_KEY = configs.get("API_KEY").data
API_BASE = configs.get("API_BASE").data
LLM_ENGINE_GPT35_16K=configs.get("LLM_ENGINE_GPT35_16K").data
ADA_MODEL=configs.get("ADA_MODEL").data

In [5]:
llm = AzureChatOpenAI(
            azure_endpoint=API_BASE,
            api_key=API_KEY,
            azure_deployment=LLM_ENGINE_GPT35_16K,
            openai_api_version=API_VERSION,
        )


In [6]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)

/usr/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The article discusses the concept of building autonomous agents using large language models (LLMs) as their core controller. It explores the components of such agent systems, including planning, memory, and tool use. The article provides examples of proof-of-concept demos and case studies, highlighting the challenges and limitations of using LLMs in autonomous agents.'

In [7]:
## Option 1. Stuff
# When we use load_summarize_chain with chain_type="stuff", we will use the StuffDocumentsChain.

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(docs))

/usr/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


The article discusses the concept of building autonomous agents powered by large language models (LLMs) as their core controllers. It explores the components of such agents, including planning, memory, and tool use. Several proof-of-concept examples, challenges, and references are provided. The limitations of finite context length, challenges in long-term planning and task decomposition, and the reliability of natural language interfaces are discussed.


In [8]:
## Option 2. Map-Reduce
#  we'll first map each document to an individual summary using an LLMChain. Then we'll use a ReduceDocumentsChain to combine those summaries into a single global summary.

from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [10]:
# Run chain
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [11]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [12]:
print(map_reduce_chain.run(split_docs))

The main themes identified across the set of documents are:

1. LLM-powered autonomous agents: The documents discuss the concept of building autonomous agents with LLM as the core controller. They highlight the potential of LLM as a general problem solver and its application in scientific discovery, generative agents simulation, and other tasks.

2. Planning and task decomposition: The importance of planning and task decomposition is emphasized. Techniques like Chain of Thought and Tree of Thoughts are mentioned for breaking down complex tasks into smaller steps. The documents also discuss the agent's ability to refine actions through self-reflection.

3. Memory utilization: The agent's utilization of short-term and long-term memory is explained. This includes the use of external vector stores for information retrieval and the integration of memory for reasoning and decision-making.

4. Tool use and external APIs: The capability of the agent to call external APIs for additional informa